# Pinecone Hybrid Search | Pinecone Hybrid Search

> [Pinecone](https://docs.pinecone.io/docs/overview)は、幅広い機能を持つベクターデータベースです。
>
> > [Pinecone](https://docs.pinecone.io/docs/overview) is a vector database with broad functionality.

このノートブックでは、内部でPineconeとHybrid Searchを使用するリトリバーの使用方法について説明します。

> This notebook goes over how to use a retriever that under the hood uses Pinecone and Hybrid Search.

このリトリーバーのロジックは、[このドキュメント](https://docs.pinecone.io/docs/hybrid-search)に基づいています

> The logic of this retriever is taken from [this documentation](https://docs.pinecone.io/docs/hybrid-search)

Pineconeを使用するには、APIキーと環境が必要です。
こちらが[インストール手順](https://docs.pinecone.io/docs/quickstart)です。

> To use Pinecone, you must have an API key and an Environment.
> Here are the [installation instructions](https://docs.pinecone.io/docs/quickstart).



In [ ]:
#!pip install pinecone-client pinecone-text

In [ ]:
import getpass
import os

os.environ["PINECONE_API_KEY"] = getpass.getpass("Pinecone API Key:")

In [75]:
from langchain.retrievers import PineconeHybridSearchRetriever

In [ ]:
os.environ["PINECONE_ENVIRONMENT"] = getpass.getpass("Pinecone Environment:")

`OpenAIEmbeddings`を使用するためには、OpenAI API Keyを取得する必要があります。

> We want to use `OpenAIEmbeddings` so we have to get the OpenAI API Key.



In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Setup Pinecone | Pineconeのセットアップ



この部分は一度だけ行う必要があるはずです。

> You should only have to do this part once.

注意：メタデータ内のドキュメントテキストを保持する「context」フィールドがインデックス化されないようにすることが重要です。現在、インデックス化したいフィールドを明示的に指定する必要があります。詳細については、Pineconeの[ドキュメント](https://docs.pinecone.io/docs/manage-indexes#selective-metadata-indexing)をチェックしてください。

> Note: it's important to make sure that the "context" field that holds the document text in the metadata is not indexed. Currently you need to specify explicitly the fields you do want to index. For more information checkout Pinecone's [docs](https://docs.pinecone.io/docs/manage-indexes#selective-metadata-indexing).



In [76]:
import os

import pinecone

api_key = os.getenv("PINECONE_API_KEY") or "PINECONE_API_KEY"
# find environment next to your API key in the Pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "PINECONE_ENVIRONMENT"

index_name = "langchain-pinecone-hybrid-search"

pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

WhoAmIResponse(username='load', user_label='label', projectname='load-test')

In [77]:
# create the index
pinecone.create_index(
    name=index_name,
    dimension=1536,  # dimensionality of dense model
    metric="dotproduct",  # sparse values supported only for dotproduct
    pod_type="s1",
    metadata_config={"indexed": []},  # see explanation above
)

作成できたので、これを使用できます

> Now that its created, we can use it



In [78]:
index = pinecone.Index(index_name)

## Get embeddings and sparse encoders | 埋め込みとスパースエンコーダーを取得する

エンベディングは密なベクトルに使用され、トークナイザーは疎なベクトルに使用されます

> Embeddings are used for the dense vectors, tokenizer is used for the sparse vector



In [79]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

テキストをスパース値にエンコードするには、SPLADEまたはBM25を選択できます。ドメイン外のタスクにはBM25の使用を推奨します。

> To encode the text to sparse values you can either choose SPLADE or BM25. For out of domain tasks we recommend using BM25.

スパースエンコーダーについての詳細は、pinecone-textライブラリの[ドキュメント](https://pinecone-io.github.io/pinecone-text/pinecone_text.html)をご覧ください。

> For more information about the sparse encoders you can checkout pinecone-text library [docs](https://pinecone-io.github.io/pinecone-text/pinecone_text.html).



In [80]:
from pinecone_text.sparse import BM25Encoder

# or from pinecone_text.sparse import SpladeEncoder if you wish to work with SPLADE

# use default tf-idf values
bm25_encoder = BM25Encoder().default()

上記のコードはデフォルトのtf-idf値を使用しています。独自のコーパスにtf-idf値を適合させることを強く推奨します。以下の方法で行うことができます：

> The above code is using default tfids values. It's highly recommended to fit the tf-idf values to your own corpus. You can do it as follow:

```python
corpus = ["foo", "bar", "world", "hello"]

# fit tf-idf values on your corpus
bm25_encoder.fit(corpus)

# store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")
```



## Load Retriever | ロード・リトリーバー

これで、リトリーバーを構築することができます！

> We can now construct the retriever!



In [81]:
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings, sparse_encoder=bm25_encoder, index=index
)

## Add texts (if necessary) | 必要に応じてテキストを追加してください

必要に応じて、テキストをリトリバーに追加することができます（まだ追加されていなければ）

> We can optionally add texts to the retriever (if they aren't already in there)



In [82]:
retriever.add_texts(["foo", "bar", "world", "hello"])

100%|██████████| 1/1 [00:02<00:00,  2.27s/it]


## Use Retriever | Retrieverの使用

これでリトリーバーを使用することができます！

> We can now use the retriever!



In [83]:
result = retriever.get_relevant_documents("foo")

In [84]:
result[0]

Document(page_content='foo', metadata={})